In [1]:
#Weight & Biases
import wandb

#Yaml
from yaml import load, FullLoader

#Embeddings
from dvats.all import *
from tsai.data.preparation import prepare_forecasting_data
from tsai.data.validation import get_forecasting_splits
from fastcore.all import *

#Dimensionality reduction
from tsai.imports import *

#Clustering
import hdbscan
import utils.config as cfg_
import time
import seaborn as sns

In [2]:
#Get W&B API
api = wandb.Api()

In [3]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="05-xai-lrp-TEST"
runname = name
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"

In [4]:
config_lrp = cfg_.get_artifact_config_xai_lrp(False)
cfg_.show_attrdict(config_lrp)

use_wandb: True
wandb_group: None
wandb_entity: mi-santamaria
wandb_project: deepvats
valid_artifact: None
train_artifact: mi-santamaria/deepvats/mvp-SWV:latest
enc_artifact: mi-santamaria/deepvats/mvp-SWV:latest
n_neighbors: 15
min_dist: 0.1
random_state: 1234
metric: euclidean
cpu_flag: False
job_type: layer_relevance_propagation
allow_val_change: True
dr_artifact: None


In [5]:
run_dr = wandb.init(
    entity           = config_lrp.wandb_entity,
    project          = config_lrp.wandb_project if config_lrp.use_wandb else 'work-nbs', 
    group            = config_lrp.wandb_group,
    allow_val_change = config_lrp.allow_val_change, 
    job_type         = config_lrp.job_type, 
    mode             = 'online' if config_lrp.use_wandb else 'disabled',
    anonymous        = 'never' if config_lrp.use_wandb else 'must',
    config           =  config_lrp,
    resume           = 'allow',
    name = runname
)
config_dr = wandb.config # Object for storing hyperparameters

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


In [7]:
config_dr.enc_artifact

'mi-santamaria/deepvats/mvp-SWV:latest'

In [8]:
config_dr.emb_artifact = 'mi-santamaria/deepvats/embeddings-SWV:latest'

In [11]:
artifacts_gettr = run_dr.use_artifact if config_dr.use_wandb else api.artifact
emb_artifact = artifacts_gettr(config_dr.emb_artifact, type='object')

In [26]:
emb_config = emb_artifact.logged_by().config

In [27]:
cfg_.show_attrdict(emb_config)

cpu: False
input_ar: mi-santamaria/deepvats/toy:v2
use_wandb: True
wandb_group: embeddings
enc_artifact: mi-santamaria/deepvats/mvp-SWV:latest
wandb_entity: mi-santamaria
wandb_project: deepvats


In [30]:
enc_artifact = artifacts_gettr(emb_config['enc_artifact'], type='learner')

In [31]:
enc_artifact

<Artifact QXJ0aWZhY3Q6NzExOTI4MjQ5>

In [24]:
enc_logger = enc_artifact.logged_by()

AttributeError: '<Run mi-santamaria/deepvats/ad865x5t (finished)>' object has no attribute 'logged_by'

In [23]:
enc_learner = enc_learner.logged_by()

NameError: name 'enc_learner' is not defined

In [ ]:
def get_embeddings(config, run_dr, config_dr, api, print_flag = False):
    # Botch to use artifacts offline
    artifacts_gettr = run_dr.use_artifact if config_dr.use_wandb else api.artifact
    enc_artifact = artifacts_gettr(config.emb_artifact, type='object')
    if print_flag: print (emb_artifact.name)
    ## TODO: This only works when you run it two timeS! WTF?
    try:
        enc_learner = enc_artifact.to_obj()
    except:
        enc_learner = enc_artifact.to_obj()
    #enc_artifact.metadata
    ## Restore artifact
    enc_logger = enc_artifact.logged_by()
    enc_artifact_train = artifacts_gettr(enc_logger.config['train_artifact'], type='dataset')
    if enc_logger.config['valid_artifact'] is not None:
        enc_artifact_valid = artifacts_gettr(enc_logger.config['valid_artifact'], type='dataset')
        if print_flag: print("enc_artifact_valid:", enc_artifact_valid.name)
    if print_flag: print("enc_artifact_train: ", enc_artifact_train.name)
    if config_dr.dr_artifact is not None:
        dr_artifact = artifacts_gettr(config_dr.dr_artifact)
    else:
        dr_artifact = enc_artifact_train
    if print_flag: print("DR artifact train: ", dr_artifact.name)
    df = dr_artifact.to_df()
    if print_flag: display(df.head())
    return df, dr_artifact, enc_artifact, enc_learner